langchain core

In [2]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool

langchain community

In [3]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain_community.tools.gmail import (
    GmailCreateDraft,
    GmailGetMessage,
    GmailGetThread,
    GmailSearch,
    GmailSendMessage,
)

langchain

In [ ]:
from langchain.agents import AgentExecutor
from langchain_classic.agents.format_scratchpad import format_to_openai_function_messages
from langchain_classic.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_classic.tools.render import render_tool_to_openai_function


In [ ]:
import os
from dotenv import load_dotenv

# 1. 基础模型导入 (保持你的原样，这是正确的)
from langchain_openai import ChatOpenAI

# 2. Gmail 工具导入 (由于是底层 API 工具，后续可能还需要配置 credentials.json，这里先完成代码架构)
from langchain_community.tools.gmail import (
    GmailCreateDraft,
    GmailGetMessage,
    GmailGetThread,
    GmailSearch,
    GmailSendMessage,
)

# 3. ⚠️ 核心改造区：引入现代化的 Agent 构建工具，抛弃所有带 "function" 的旧包
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# 必备起手式
load_dotenv(override=True)

# 初始化大模型 (注意：做 Agent 任务必须用 Chat 模型，且温度最好设为 0 以保证严谨)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 把你的 5 个 Gmail 工具打包成一个列表
tools = [
    GmailCreateDraft(),
    GmailGetMessage(),
    GmailGetThread(),
    GmailSearch(),
    GmailSendMessage()
]

# =================================================================
# 🚀 见证 LCEL 的魔法：现代 Agent 的三步构建法
# =================================================================

# 步骤 1：构建专门给 Agent 看的提示词模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个极其得力的邮件助手。你可以帮我读取、搜索、起草和发送 Gmail 邮件。"),
    ("human", "{input}"),
    # 🌟 极其关键的占位符：用来存放 Agent 的“内心独白”和工具调用的中间结果
    ("placeholder", "{agent_scratchpad}"), 
])

# 步骤 2：一键生成支持 Tool Calling 的现代 Agent
# 这一行代码直接替代了老师那 3 行极其复杂的 format_scratchpad 和 output_parser 组装！
agent = create_tool_calling_agent(llm, tools, prompt)

# 步骤 3：将 Agent 和 Tools 塞进执行器 (Executor)
# verbose=True 会在控制台打印出 Agent 的思考过程，非常震撼！
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("✅ 现代版 Gmail Agent 执行器已完美构建！\n")

# =================================================================
# 测试运行 (需要你本地配置好 Gmail API 的 token.json 才能真正跑通)
# =================================================================
# 如果你的 Gmail API 环境没问题，取消下面两行的注释就可以让 AI 帮你读邮件了！
# response = agent_executor.invoke({"input": "帮我搜索一下收件箱里有没有关于 'DSSI 科研' 的邮件？"})
# print(response["output"])